In [1]:
import pandas as pd
import glob
import math

In [53]:
pd.set_option('display.max_columns', 800)

In [2]:
train_set_dir = "/media/data2/Data/wsdm2019/data/training_set/"
test_set_dir = "/media/data2/Data/wsdm2019/data/test_set/"
track_features_dir = "/media/data2/Data/wsdm2019/python/data/track_features/"

# output_dir_csv = "/media/data2/Data/wsdm2019/python/data/training_set/"
output_dir_pkl = "/media/data2/Data/wsdm2019/python/data/test_examples/"

In [4]:
test_files = sorted(glob.glob(test_set_dir+"*.csv"))

In [22]:
test_files_history = test_files[66:]
test_files_predict = test_files[:66]

In [23]:
bool_columns = ['skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch', 'no_pause_before_play', 'short_pause_before_play',
                'long_pause_before_play', 'hist_user_behavior_is_shuffle', 'premium']
numerical_columns = ['hist_user_behavior_n_seekfwd', 'hist_user_behavior_n_seekback']
numerical_columns_range_map = {'hist_user_behavior_n_seekfwd': (0,100), 'hist_user_behavior_n_seekback': (0,100)}
dropped_columns = ['session_position', 'session_length', 'hour_of_day', 'date'] # temporaryly dropped, will add back if useful
categorical_columns = ['context_type', 'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end']
categorical_columns_map = {'context_type':
                           {'catalog': 0, 'charts': 1, 'editorial_playlist': 2, 'personalized_playlist': 3, 'radio': 4, 'user_collection': 5},
                           'hist_user_behavior_reason_start':
                           {'appload': 0, 'backbtn': 1, 'clickrow': 2, 'endplay': 3, 'fwdbtn': 4, 'playbtn': 5, 'popup': 6, 'remote': 7, 'trackdone': 8, 'trackerror': 9, 'uriopen': 10, 'clickside': 11},
                            'hist_user_behavior_reason_end':
                           {'appload': 0, 'backbtn': 1, 'clickrow': 2, 'endplay': 3, 'fwdbtn': 4, 'logout': 5, 'popup': 6, 'remote': 7, 'trackdone': 8, 'uriopen': 9, 'clickside': 10}
}
id_column = ['session_id', 'track_id_clean']

In [24]:
import pickle
with open (track_features_dir + "track2idx_all.pkl", "rb") as f:
    track2idx = pickle.load(f)

In [25]:
example_test_history = test_files_history[0]
example_test_predict = test_files_predict[0]

In [26]:
df_hist = pd.read_csv(example_test_history)
df_predict = pd.read_csv(example_test_predict)

In [40]:
df_hist_copy = df_hist.copy(deep=True)
df_predict_copy = df_predict.copy(deep=True)

In [54]:
def convert_features(new_df):
    df_features = new_df.drop(['session_id', 'track_id_clean'], axis=1)
    for feature in df_features.columns:
        if feature in bool_columns:
            # for all the bool features, convert to integer [0 --> False, 1--> True]
            new_df[feature] = new_df[feature].astype(int) 
        elif feature in numerical_columns:
            # for numerical columns, define min and max for processing
            # do min-max scaling which is (x - min) / (max - min), if value is bigger than max set to max
            (min, max) = numerical_columns_range_map[feature]
            new_df[feature] = new_df[feature].clip(min, max)
            new_df[feature] = (new_df[feature] - min) / (max - min)
        elif feature in categorical_columns:
            # if feature is categorical, convert feature into a map of item --> integer, and leave integer 0 for padding as bool
    #         feature_map, feature_length = convert_feature_to_map(new_df[feature].drop_duplicates().sort_values().values)
            new_df[feature] = new_df[feature].map(categorical_columns_map[feature])
        elif feature in dropped_columns:
            new_df = new_df.drop(feature, axis=1)

    # sanity check
    print("if there is nan: " + str(new_df.isnull().values.any()))
    
    return new_df

In [56]:
df_hist_converted = convert_features(df_hist_copy)

if there is nan: False


In [55]:
df_hist.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00001e52-55c0-470c-8d32-85c691c2019a,1,16,t_a0b2a533-caad-4a72-af14-e647bd7e9c1f,False,True,True,False,0,0,0,0,1,0,False,21,2018-07-14,False,catalog,appload,endplay
1,0_00001e52-55c0-470c-8d32-85c691c2019a,2,16,t_a998907c-e11c-4a4e-b9d2-b34846bfb839,False,False,True,False,1,1,0,0,0,0,False,21,2018-07-14,False,user_collection,clickrow,endplay
2,0_00001e52-55c0-470c-8d32-85c691c2019a,3,16,t_977b3ee2-e7a8-4357-b164-f663446c1a25,False,False,False,True,0,0,0,1,0,1,False,21,2018-07-14,False,user_collection,clickrow,trackdone
3,0_00001e52-55c0-470c-8d32-85c691c2019a,4,16,t_e2dda38d-f826-481b-b49a-a1d9051f5c6d,False,True,True,False,0,0,1,1,0,0,False,21,2018-07-14,False,user_collection,trackdone,fwdbtn
4,0_00001e52-55c0-470c-8d32-85c691c2019a,5,16,t_b10f5a2a-ee7b-47c3-9789-4ce0f96c6989,True,True,True,False,0,1,0,0,0,0,False,21,2018-07-14,False,user_collection,fwdbtn,fwdbtn


In [57]:
df_hist_converted.head()

,session_id,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00001e52-55c0-470c-8d32-85c691c2019a,t_a0b2a533-caad-4a72-af14-e647bd7e9c1f,0,1,1,0,0,0,0,0,0.01,0.00,0,0,0,0,3
1,0_00001e52-55c0-470c-8d32-85c691c2019a,t_a998907c-e11c-4a4e-b9d2-b34846bfb839,0,0,1,0,1,1,0,0,0.00,0.00,0,0,5,2,3
2,0_00001e52-55c0-470c-8d32-85c691c2019a,t_977b3ee2-e7a8-4357-b164-f663446c1a25,0,0,0,1,0,0,0,1,0.00,0.01,0,0,5,2,8
3,0_00001e52-55c0-470c-8d32-85c691c2019a,t_e2dda38d-f826-481b-b49a-a1d9051f5c6d,0,1,1,0,0,0,1,1,0.00,0.00,0,0,5,8,4
4,0_00001e52-55c0-470c-8d32-85c691c2019a,t_b10f5a2a-ee7b-47c3-9789-4ce0f96c6989,1,1,1,0,0,1,0,0,0.00,0.00,0,0,5,4,4


In [60]:
df_predict_converted = convert_features(df_predict_copy)

if there is nan: False


In [61]:
df_predict_converted.head()

,session_id,track_id_clean
0,0_00001e52-55c0-470c-8d32-85c691c2019a,t_2e6b3753-5b16-4aa2-97f8-c77a81377139
1,0_00001e52-55c0-470c-8d32-85c691c2019a,t_fb588cbd-afee-4155-86a4-f8715942de28
2,0_00001e52-55c0-470c-8d32-85c691c2019a,t_0303b567-f375-430c-9457-d41ccfbcdc59
3,0_00001e52-55c0-470c-8d32-85c691c2019a,t_be6832af-d395-4111-9a0b-ac124a71e174
4,0_00001e52-55c0-470c-8d32-85c691c2019a,t_a998907c-e11c-4a4e-b9d2-b34846bfb839


In [28]:
df_predict.head()

,session_id,track_id_clean,session_position,session_length
0,0_00001e52-55c0-470c-8d32-85c691c2019a,t_2e6b3753-5b16-4aa2-97f8-c77a81377139,9,16
1,0_00001e52-55c0-470c-8d32-85c691c2019a,t_fb588cbd-afee-4155-86a4-f8715942de28,10,16
2,0_00001e52-55c0-470c-8d32-85c691c2019a,t_0303b567-f375-430c-9457-d41ccfbcdc59,11,16
3,0_00001e52-55c0-470c-8d32-85c691c2019a,t_be6832af-d395-4111-9a0b-ac124a71e174,12,16
4,0_00001e52-55c0-470c-8d32-85c691c2019a,t_a998907c-e11c-4a4e-b9d2-b34846bfb839,13,16


In [62]:
df_group_hist = df_hist_converted.groupby('session_id')
df_group_predict = df_predict_converted.groupby('session_id')

In [73]:
print(df_group_predict.ngroups)

444491


In [74]:
print(df_group_hist.ngroups)

444491


In [76]:
ngroups = df_group_hist.ngroups

In [77]:
examples = []
counter = 0

for name, group_history in df_group_hist:
    group_predict = df_group_predict.get_group(name) 
    
    # extract tracks data
    tracks_history = [track2idx[a] for a in group_history['track_id_clean'].values]
    tracks_predict = [track2idx[a] for a in group_predict['track_id_clean'].values]
    
    # extract bool data
    tracks_feature_bool = group_history[bool_columns].values
    
    # extract numerical data
    tracks_feature_numerical = group_history[numerical_columns].values
    
    # extract categorical data
    tracks_feature_categorical = group_history[categorical_columns].values
    
    examples.append({
        'tracks_history': tracks_history,
        'tracks_predict': tracks_predict,
        'tracks_feature_bool': tracks_feature_bool,
        'tracks_feature_numerical': tracks_feature_numerical,
        'tracks_feature_categorical': tracks_feature_categorical
    })
    
    counter += 1
    if (counter % 5000) == 0:
        print( counter / ngroups)

# sanity check
print(len(examples))

0.011248821685928398
0.022497643371856797
0.0337464650577852
0.04499528674371359
0.056244108429642
0.0674929301155704
0.0787417518014988
0.08999057348742719
0.10123939517335559
0.112488216859284
0.12373703854521238
0.1349858602311408
0.14623468191706918
0.1574835036029976
0.16873232528892598
0.17998114697485437
0.1912299686607828
0.20247879034671118
0.21372761203263957
0.224976433718568
0.23622525540449638
0.24747407709042477
0.2587228987763532
0.2699717204622816
0.28122054214820996
0.29246936383413835
0.3037181855200668
0.3149670072059952
0.3262158288919236
0.33746465057785197
0.34871347226378036
0.35996229394970874
0.3712111156356372
0.3824599373215656
0.39370875900749397
0.40495758069342236
0.41620640237935075
0.42745522406527914
0.4387040457512076
0.449952867437136
0.46120168912306436
0.47245051080899275
0.48369933249492114
0.49494815418084953
0.5061969758667779
0.5174457975527064
0.5286946192386347
0.5399434409245631
0.5511922626104916
0.5624410842964199
0.5736899059823484
0.58493

In [78]:
print(examples[0])

{'tracks_history': [2491238, 1560519, 3683392, 3102061, 3681611, 1560519, 3683392, 3102061], 'tracks_predict': [1583261, 568640, 3426672, 211679, 1560519, 1515786, 2209695, 373938], 'tracks_feature_bool': array([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0]]), 'tracks_feature_numerical': array([[0.01, 0.  ],
       [0.  , 0.  ],
       [0.  , 0.01],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ]]), 'tracks_feature_categorical': array([[0, 0, 3],
       [5, 2, 3],
       [5, 2, 8],
       [5, 8, 4],
       [5, 4, 4],
       [5, 4, 3],
       [5, 2, 4],
       [5, 4, 3]])}


In [83]:
example_test_history[56:-4]

'20180715_000000000000'